In [2]:
# armazenar os documentos totais em variavel

import os
#rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/20_newsgroup'
rootdir = '/media/developer/DATA/doutorado/rec informacao/20_newsgroup'
docs = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #if os.path.basename(subdir) in ['alt.atheism', 'rec.sport.baseball']:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [8]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

#stemmer = PorterStemmer()
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
def remove_header(text):
    # Find the first occurrence of '\n\n'
    pos = text.find('\n\n')

    # Remove everything after the first '\n\n'
    if pos >= 0:
        text = text[pos+2:]
    return text
    

In [10]:
def remove_email(text):
    pattern = r'\S+@\S+'
    # Remove email addresses from the string
    text = re.sub(pattern, '', text)
    return text


In [11]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):


    #text = remove_header(text)

    #text = remove_email(text)


    # Converter tudo para letras minúsculas
    text = text.lower()
    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)
    
    # manter so texto
    #text = re.sub('[^A-Za-z ]+', '', text)

    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 2]

    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    #words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)
  
    return text

In [12]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    for document in docs:
        tema = document['tema']
        nome_arquivo = document['nome_arquivo']
        conteudo = document['conteudo']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
            'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [13]:
preprocessed_documents = preprocess_collection(docs)
frases = [o['conteudo'] for o in preprocessed_documents]

In [15]:
frases[0]

"xref cantaloupe.srv.cs.cmu.edu talk.abortion:120737 alt.atheism:53341 talk.religion.misc:83726 path cantaloupe.srv.cs.cmu.edu magnesium.club.cc.cmu.edu news.sei.cmu.edu cis.ohio-state.edu pacific.mps.ohio-state.edu zaphod.mps.ohio-state.edu usc sol.ctr.columbia.edu ira.uka.de germany.eu.net thoth.mchp.sni.de horus.ap.mchp.sni.de d012s658 frank frank d012s658.uucp frank o'dwyer newsgroups talk.abortion alt.atheism talk.religion.misc subject 2000 year say christian morality date apr 1993 20:00:08 gmt organization siemens-nixdorf line message-id reference nntp-posting-host d012s658.ap.mchp.sni.de article mathew writes frank d012s658.uucp frank o'dwyer writes article mathew mantis.co.uk mathew writes ask think wrong relativism correct misconception well cut chase admit find least attractive realtivism elevates heinous level good saying effect good n't good thus terrorist elevated level man peace complete nonsense relativism mean saying absolute standard morality mean saying standard moral

In [46]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in frases]

bm25 = BM25Okapi(tokenized_corpus)

In [47]:
# armazenar os documentos totais em variavel

import os
#rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/mini_newsgroups'
rootdir = '/media/developer/DATA/doutorado/rec informacao/mini_newsgroups'

docs_consultas = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs_consultas.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [48]:
dirs_abs = []
for subdir, dirs, files in os.walk(rootdir):
    if os.path.basename(subdir) != '20_newsgroup':
        dirs_abs.append(os.path.basename(subdir))

temas = {}
for d in dirs_abs:
    temas[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(temas)    

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'alt.atheism': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.graphics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.os.ms-windows.misc': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.ibm.pc.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.mac.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.windows.x': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'misc.forsale': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.autos': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.motorcycles': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.baseball': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.hockey': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.crypt': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.electronics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 

In [49]:
def calcula_map(ranking, tema):
    acertos = 0
    maps = []
    for i, doc in enumerate(ranking):
        if tema in doc:
            acertos +=1
            maps.append(acertos/(i+1))
    if len(maps) > 0:
        return mean(maps)
    else:
        print("não achou nada")
        return 0.0

In [50]:
doc_consultas_preproc = preprocess_collection(docs_consultas)

In [42]:
doc_consultas_preproc[0]['conteudo']

"newsgroups alt.atheism path cantaloupe.srv.cs.cmu.edu das-news.harvard.edu noc.near.net howland.reston.ans.net zaphod.mps.ohio-state.edu darwin.sura.net haven.umd.edu uunet mnemosyne.cs.du.edu nyx jcopelan jcopelan nyx.cs.du.edu one subject message-id sender usenet mnemosyne.cs.du.edu netnews admin account organization salvation army draft board reference date fri apr 13:38:30 gmt line article geoff east.sun.com writes posting provoked checking save file memorable post first captured ken arromdee feb 1990 subject atheist article 473 question article 53766 average article day last three year others noted current posting rate kill file depressing large among posting saved early day article following notable loren sunlight.llnl.gov loren petrich jchrist nazareth.israel.rel jesus christ nazareth mrc tomobiki-cho.cac.washington.edu mark crispin perry apollo.hp.com jim perry lippard uavax0.ccit.arizona.edu james lippard minsky media.mit.edu marvin minsky interesting bunch wonder n't hear ad

In [37]:
# apenas para teste
tokenized_query = doc_consultas_preproc[101]['conteudo'].split(" ")
bm25.get_top_n(tokenized_query, frases, n=100)

In [56]:
from statistics import mean

from IPython.display import clear_output

# consultando
p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""

for c, doc_consulta in enumerate(doc_consultas_preproc):
    clear_output(wait=True)
    print(c)
    #ranking = similaridade(doc_consulta['conteudo'], preprocessed_documents, idfs)
    tokenized_query = doc_consulta['conteudo'].split(" ")
    ranking = bm25.get_top_n(tokenized_query, frases, n=len(doc_consultas_preproc))
    acertos = 0
    
    map_f.append(calcula_map(ranking, doc_consulta['tema']))
    
    for i, doc in enumerate(ranking):
        #print(f"Documento {doc[0]} - Target {doc[1]} -->  {doc[2]} - Similaridade: {doc[3]}")
        #if doc[1] == doc_consulta['tema']:
        
        if doc_consulta['tema'] in doc:
            #print(f"{doc_consulta['tema']} e {doc}")
            acertos+=1
        if i+1 == 10:
            p10.append(acertos/10.0)            
        if i+1 == 20:
            p20.append(acertos/20.0)
        if i+1 == 50:
            p50.append(acertos/50.0)
        if i+1 == 100:
            p100.append(acertos/100.0) 
        #map.append(calcula_map(ranking, doc_consulta['tema']))
    if c > 0:
        if tema_atual != doc_consulta['tema'] or c == len(doc_consultas_preproc) - 1:
            temas[tema_atual]['p10'] = mean(p10)
            temas[tema_atual]['p20'] = mean(p20)
            temas[tema_atual]['p50'] = mean(p50)
            temas[tema_atual]['p100'] = mean(p100)
            temas[tema_atual]['map'] = mean(map_f)
            p10 = []
            p20 = []
            p50 = []
            p100 = []
            map = []

    tema_atual = doc_consulta['tema']
    # if c > 202:
    #     break    


1999


In [57]:
temas

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'alt.atheism': {'p10': 0.9168316831683169,
  'p20': 0.8673267326732673,
  'p50': 0.7956435643564357,
  'p100': 0.7310891089108911,
  'map': 0.5172430453536131},
 'comp.graphics': {'p10': 0.591,
  'p20': 0.525,
  'p50': 0.4802,
  'p100': 0.4395,
  'map': 0.4223569630118447},
 'comp.os.ms-windows.misc': {'p10': 0.6940000000000001,
  'p20': 0.613,
  'p50': 0.5488,
  'p100': 0.5048,
  'map': 0.4138586312336424},
 'comp.sys.ibm.pc.hardware': {'p10': 0.65,
  'p20': 0.5885,
  'p50': 0.5058,
  'p100': 0.4549,
  'map': 0.3974239513975844},
 'comp.sys.mac.hardware': {'p10': 0.69,
  'p20': 0.6005,
  'p50': 0.5072,
  'p100': 0.4632,
  'map': 0.39035198237358854},
 'comp.windows.x': {'p10': 0.6930000000000001,
  'p20': 0.614,
  'p50': 0.5542,
  'p100': 0.5003,
  'map': 0.383269097619498},
 'misc.forsale': {'p10': 0.58,
  'p20': 0.516,
  'p50': 0.4788,
  'p100': 0.4482,
  'map': 0.38428422304347404},
 'rec.autos': {'p10': 0.76

In [58]:
import json
with open('data_lemma_bm25_2.json', 'w') as f:
    json.dump(temas, f)

In [59]:
import pandas as pd

df = pd.DataFrame(temas)
df =df.transpose()
df.drop('mini_newsgroups')


,p10,p20,p50,p100,map
alt.atheism,0.916832,0.867327,0.795644,0.731089,0.517243
comp.graphics,0.591000,0.525000,0.480200,0.439500,0.422357
comp.os.ms-windows.misc,0.694000,0.613000,0.548800,0.504800,0.413859
comp.sys.ibm.pc.hardware,0.650000,0.588500,0.505800,0.454900,0.397424
comp.sys.mac.hardware,0.690000,0.600500,0.507200,0.463200,0.390352
comp.windows.x,0.693000,0.614000,0.554200,0.500300,0.383269
misc.forsale,0.580000,0.516000,0.478800,0.448200,0.384284
rec.autos,0.763000,0.682000,0.590600,0.511500,0.384422
rec.motorcycles,0.836000,0.755000,0.653200,0.561800,0.388852
rec.sport.baseball,0.837000,0.786000,0.694200,0.612600,0.394168


In [1]:
df.describe()

NameError: name 'df' is not defined